In [ ]:
from collections import Counter
import gzip

`mutyper` demo
==

This notebook demonstrates how to use the Python package [`mutyper`](https://github.com/harrispopgen/mushi/tree/master/mutyper): a utility for... 

## Computing mutation type data to VCF/BCF data

- Studying mutation spectra usually begins with computing the mutation type for SNPs in a VCF/BCF file
- To polarize SNPs from REF/ALT to ancestral/derived and determine their local context, we need an estimate of the ancestral sequence, which usually takes the form of a FASTA file.
- `mutyper` is a Python package for doing this
- it also has a command-line interface for easily integrating in more complex bioinformatic pipelines.

### Python API demo
Path to an ancestral FASTA file for human chromosome 1

In [ ]:
fasta = 'data/human_ancestor_GRCh37_e59/human_ancestor_1.fa'

The `mutyper` package has a single module `ancestor` containing a single class `Ancestor`. We now import it and instantiate an `Ancestor` object using our FASTA file, and tell it we're interested in 3-mer context (by default it will be triplet/3-mer)

In [ ]:
from mutyper.ancestor import Ancestor

ancestor = Ancestor(fasta, k=3)

We can access the `fasta` attribute to inspect FASTA record ids

In [ ]:
ancestor.fasta.keys()

In [ ]:
chrom = 'ANCESTOR_for_chromosome:GRCh37:1:1:249250621:1'

In [ ]:
ancestor.fasta[chrom]

and we can Pythonically slice sequences via fast random access without loading into memory ([`pyfaidx`](https://pythonhosted.org/pyfaidx/) under the hood):

In [ ]:
start = 20000000
end = 20000100
ancestor.fasta[chrom][start:end]

We can also access these `FastaRecord` slices as string-like biopython `Seq` objects

In [ ]:
ancestor.fasta[chrom][start:end].seq

**The `mutation_type` method** allows us to specify a SNP by the usual CHROM, POS, REF, ALT, and returns the correctly polarized mutation type as a tuple of ancestral kmer and derived kmer.

In [ ]:
ancestor.mutation_type(chrom, 20000000, 'A', 'T')

In [ ]:
ancestor.mutation_type(chrom, 20000000, 'T', 'A')

infinite sites violation (i.e. not biallelic)

In [ ]:
ancestor.mutation_type(chrom, 20000000, 'T', 'C')

**The `region_context` method** returns a generator of ancestral contexts over the positions in a region specified as in BED file format CHROM, START, END

In [ ]:
start = 20000105
end = 20000115
for context in ancestor.region_context(chrom, start, end):
    print(context, end=' ')

FASTA sites that are not capital ACGT are ambiguous, so context is `None`. 

We can use this generator method to lazily compute the masked genomic target size for each context (which may be useful for normalizing spectra in different regions, or calibrating mutation rates). As an example here's the path to a BED mask file (based on the 1000 Genomes strict mask).

In [ ]:
bed = '1KG/scons_output/mask.bed.gz'

Loop over BED file entries and update a `Counter` object for the different triplets (we'll just do the first 1000 entries)

In [ ]:
target_sizes = Counter()
for ct, line in enumerate(gzip.open(bed, 'rt')):
    this_chrom, start, end = line.split('\t')
    target_sizes.update(ancestor.region_context(chrom, int(start), int(end)))
    if ct > 1000:
        break
print(target_sizes)

### Command line interface demo

Path to a BCF file for chromosome 22 from the 1000 Genomes Project.

In [ ]:
bcf = 'data/phase3_1000genomes/bcfs/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5.20130502.genotypes.bcf'

Here's a bash cell (not python, due to the `!`) that pipes SNP data filtered with [`bcftools view`](http://samtools.github.io/bcftools/bcftools.html#view) into **the [`mutyper.ancestor`](https://github.com/harrispopgen/mushi/blob/master/mutyper/ancestor.py) program** to add mutation_type to the INFO field. (for speed will just look at a 1kb region). This program takes an optional argument `k` to define kmer size (3 by default). We then pipe to [`bcftools query`](http://samtools.github.io/bcftools/bcftools.html#query) to show that the INFO field now contains mutation_type

In [ ]:
!bcftools view -T {bed} -r1:100000000-101000000 -m2 -M2 -v snps -c 1:minor -Ou -f PASS -U {bcf} \
 | python mutyper/ancestor.py {fasta} \
 | bcftools query -f '%CHROM\t%POS\t%REF\t%ALT\t%INFO/mutation_type\n' | head -5

We can compute ancestral sequence content in a pipelined fashion too. We use [`tabix`](http://www.htslib.org/doc/tabix.html) to filter the gzipped BED mask file to a region of interest (first 10Mb of chromosome 1), and pipe that to **the [`mutyper.masked_size`](https://github.com/harrispopgen/mushi/blob/master/mutyper/masked_size.py) program** to print the triplet-wise target sizes in tabular format. This program also takes an optional argument `k`.

In [ ]:
!tabix -p bed {bed} 1:100000000-101000000 \
 | python mutyper/masked_size.py {fasta} | head

Finally, we can pipe BCF/VCF data containing mutation_type data in the INFO field (see above) to **the program [`mutyper.ksfs`](https://github.com/harrispopgen/mushi/blob/master/mutyper/ksfs.py)** to compute a $k$-SFS in tabular form. The $k$-SFS is a matrix of the count of each mutation type (columns) in the input SNP data at each allele frequency (rows). This can also be used to compute mutation spectra of individuals.

**A single individual's mutation spectrum** (filter the BCF to sample HG00096):

In [ ]:
!bcftools view -T {bed} -r1:100000000-101000000 -m2 -M2 -v snps -c 1:minor -Ou -f PASS -U {bcf} \
 | bcftools view -s HG00096 -c 1:minor \
 | python mutyper/ancestor.py {fasta} | python mutyper/ksfs.py \
 | cut -f-10

**$k$-SFS of all individuals in the BCF**

In [ ]:
!bcftools view -T {bed} -r1:100000000-101000000 -m2 -M2 -v snps -c 1:minor -Ou -f PASS -U {bcf} \
 | python mutyper/ancestor.py {fasta} | python mutyper/ksfs.py \
 | head -10 | cut -f-10